#  Отчет по практическому заданию №2 
## Постановка задачи

   Написать и отладить процедуру, вычисляющую определенный интеграл от функции одного действительного переменного по квадратурной формуле Симпсона. Подпрограмма должна обеспечивать автоматический контроль точности и деление интервала интегрирования на подынтервалы там, где это необходимо.
   
   
## Вычисление интеграла по квадратурной формуле Симпсона 

Формула Симпсона основана на замене подынтегральной функции f(x) на отрезке [a, b] дугой параболы, т.е. функция f(x) аппроксимируется параболой вида: $P(x)=\alpha x^2 + \beta x + \gamma$.

Разобъем отрезок $[a,b]$ на четное число равных отрезков $n = 2m$, при этом точки $x_0, x_2, x_4, ... , x_{n-2}, x_n$ - точки деления ($x_0 = a, x_n = b$). Обозначим через $x_1, x_3, x_5$, ... середины отрезков $[x_0, x_2], [x_2, x_4], [x_4, x_6]$ и т.д. Применив для каждого отрезка разбиения элементарную формулу Симпсона, получим формулу парабол.

### $\displaystyle{I ~ = ~\int\limits_a^b f(x) dx \approx \frac{h}{3} ~ [y_0 + y_n + 2(y_{2}+y_4+...+y_{n-2})+4(y_1+y_3+...+y_{n-1})]}$ 

## Оценка точности вычисления

Интеграл вычисляется по формуле парабол Симпсона при числе шагов, равном n, а затем при числе шагов, равном 2n. Погрешность вычисления значения интеграла при числе шагов, равном 2n, определяется по формуле Рунге:
${\displaystyle \Delta \approx \frac{1}{15} |I_{2n}-I_{n}|}$

Таким образом, интеграл вычисляется для последовательных значений числа шагов ${\displaystyle N = n, 2n, 4n,\dots }$ , где ${\displaystyle n}$ — начальное число шагов. 

Процесс вычислений заканчивается, когда для очередного значения N будет выполнено условие ${\displaystyle \Delta <\varepsilon  }$, где ${\displaystyle \varepsilon }$  — заданная точность.

### Введем начальные данные $a, b, E, y1$

In [2]:
from sympy import *
import numpy as np


a=0    # нижний предел интегрирования
b=1     # верхний предел интегрирования
E = 0.00001  #задаем точность вычисления

x1 = symbols('x')        # задаем функцию, y1 для подсчета производной, f(x) для нахождения интеграла
y1=1/(x1+1) 
print(y1)
def f(x):
    return 1/(float(x)+1)

1/(x + 1)


In [3]:
n=4    # исходное кол-во интервалов разбиения 
h = (b-a)/n       # интервал 
h2 = (b-a)/(2*n) 
x=np.arange(a, b+h, h) 
x2=np.arange(a, b+h2, h2) 
print(x)
print(x2)

[0.   0.25 0.5  0.75 1.  ]
[0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.   ]


In [4]:
c2 = c4 = c22 = c24 = 0
                            
for i in range(1, int((n)/2)):
    c2 += f(x[2*i])   
    c4 += f(x[2*i-1])
c4 += f(x[n-1])    

I = h/3 * (f(x[0])+f(x[n])+2*c2+4*c4)

for i in range(1, int(n)):    
    c22 += f(x2[2*i]) 
    c24 += f(x2[2*i-1])
    
c24 += f(x2[2*n-1])

Ik = h2/3 * (f(x2[0])+f(x2[2*n])+2*c22+4*c24)

print(I,Ik)

0.6932539682539682 0.6931545306545306


### Сравниваем значения интегралов при n, 2n, 4n ...

In [5]:
k=2*n

while abs(Ik-I)/15>E:
    I=Ik
    c2=0
    c4=0
    k = k*2
    h=(b-a)/k
    x=np.arange(a, b+h, h)
    
    for i in range(1, int(k/2)):    
        c2 += f(x[2*i]) 
        c4 += f(x[2*i-1])
    c4 += f(x[k-1]) 
    
    Ik = (b-a)/(k*3) * (f(x[0])+f(x[k])+2*c2+4*c4)
print(k,h)
print(I, '+-' , abs(Ik-I)/15)

8 0.25
0.6932539682539682 +- 6.629173295843079e-06


## Таким образом, $I = 0.693254  \pm 0.000007 $

## Второй способ вычисления интервала для требуемого уровня точности

$ \displaystyle{R \simeq - \frac{(b-a) ~ h^4 ~ f^{IV}_{max}}{180}} $   $~~~$ - $~~~$  остаточный член

Для получения точности $\varepsilon$ нужен шаг 

### ${\displaystyle h_{\varepsilon} < \left( \frac{180 \varepsilon }{(b-a) f^{IV}_{max}} \right) ^{1/4}   }$






### Поиск четвертой производной

In [6]:
pr4 = diff(y1,x1,4)  # взяли четвертую производную по х

print(pr4)

24/(x + 1)**5


### Поиск максимума четвертой производной на интервале $[a,b]$

In [7]:
from sympy import solveset, symbols, Interval, Min, Max
x = symbols('x')

lower_bound = a
upper_bound = b

f = pr4

zeros = solveset(f, x, domain=Interval(lower_bound, upper_bound))
assert zeros.is_FiniteSet # If there are infinite solutions the next line will hang.
res_min = Min(f.subs(x, lower_bound), f.subs(x, upper_bound), *[f.subs(x, i) for i in zeros])
res_max = Max(f.subs(x, lower_bound), f.subs(x, upper_bound), *[f.subs(x, i) for i in zeros])
print(res_max)

24


In [8]:
h_E=(180*E/(b-a)/res_max)**(1/4)
print(h_E,h)

0.0930604859102100 0.25


### Результат второго метода ${\displaystyle h_{\varepsilon} = 0.093}$ меньше, чем резульат первого метода ${\displaystyle h = 0.25}$ 

### Остаточный член

In [9]:
R = - (b-a)/180*h**4*res_max
print(R)

-0.000520833333333333
